In [ ]:
import sys,os
import torch
import yaml
import logging
from pydantic import ValidationError

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from datasets.weather_bench import WeatherDataset
from models.CiT.datasets.dataset import CustomDataset
from models.CiT.training.configs import TrainingConfig
from models.CiT.training.configs import TrainingRunConfig

def get_normal_dataset(config: TrainingConfig):
    device = ("cuda" if torch.cuda.is_available() else "cpu" )
    device = torch.device(device)

    weather = WeatherDataset(config.train_offset, device=device)
    # dataset.shape:  torch.Size([7309, 100, 1450])
    source, label, mean_std = weather.load(config.air_variable, config.surface_variable)
    dataset = CustomDataset(source, label, config.max_lead_time)
    var_list = weather.get_var_code(config.air_variable, config.surface_variable)
    print(mean_std.shape)
    return dataset, mean_std, var_list

config_path = os.path.join(os.path.dirname(os.getcwd()), 'configs/train_config.yaml')

try:
    with open(config_path) as f:
        config_dict = yaml.safe_load(f)
    config: TrainingRunConfig = TrainingRunConfig.parse_obj(config_dict)
except FileNotFoundError:
    logging.error(f"Config file {config_path} does not exist. Exiting.")
except yaml.YAMLError:
    logging.error(f"Config file {config_path} is not valid YAML. Exiting.")
except ValidationError as e:
    logging.error(f"Config file {config_path} is not valid. Exiting.\n{e}")


dataset, _, _ = get_normal_dataset(config.training)

In [ ]:
%load_ext autoreload
%autoreload 2

from models.CiT.training.lightning import TrainModule

model = TrainModule.load_from_checkpoint('/workspace/Haea_dev/models/CiT/tb_logs/lightning_logs/0cw4rfha/checkpoints/epoch=69-step=153370.ckpt')

In [ ]:
from torch.utils.data import DataLoader, Dataset
import tqdm

data_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8)
att_maps = []
epoch = 

for i, batch in enumerate(tqdm.tqdm(data_loader)):
    att_map = model.model.get_attention_maps(batch.to(device)).cpu().detach()
    att_maps.append(att_map)